## TP2

### Partie 1

In [1]:
import pandas as pd

In [5]:
df_ratings = pd.read_csv('C:/Users/leo-a/Desktop/Cours M2/Apprentissage pour le web/TP2/ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])

movies_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL'] + [f'genre_{i}' for i in range(19)]
df_movies = pd.read_csv('C:/Users/leo-a/Desktop/Cours M2/Apprentissage pour le web/TP2/ml-100k/u.item', sep='|', names=movies_cols, encoding='latin-1')

In [16]:
df_movies.head()

,movie_id,title,release_date,video_release_date,IMDb_URL,genre_0,genre_1,genre_2,genre_3,genre_4,...,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17,genre_18
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


### Partie 2

In [9]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

In [8]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_ratings[['user_id', 'movie_id', 'rating']], reader)

In [10]:
train, test = train_test_split(data, test_size=0.2)

In [11]:
from surprise import KNNBasic
sim_options = {
    'name': 'cosine',
    'user_based': True  
}
algo = KNNBasic(sim_options=sim_options)
algo.fit(train)


Computing the cosine similarity matrix...
Done computing similarity matrix.


In [13]:
from surprise import accuracy
predictions = algo.test(test)
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)


RMSE: 1.0186
MAE:  0.8046


### Partie 3

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [18]:

genres_columns = df_movies.columns[5:]

df_movies['genres'] = df_movies[genres_columns].apply(lambda row: ' '.join([col for col, val in zip(genres_columns, row) if val == 1]), axis=1)

print(df_movies[['title', 'genres']].head())


               title                    genres
0   Toy Story (1995)   genre_3 genre_4 genre_5
1   GoldenEye (1995)  genre_1 genre_2 genre_16
2  Four Rooms (1995)                  genre_16
3  Get Shorty (1995)   genre_1 genre_5 genre_8
4     Copycat (1995)  genre_6 genre_8 genre_16


In [20]:

tfidf = TfidfVectorizer()

tfidf_matrix = tfidf.fit_transform(df_movies['genres'])

print(tfidf_matrix.shape)


(1682, 19)


In [21]:
all_genres = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 
              'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

user_genres = input(f"Veuillez entrer vos genres préférés, séparés par des virgules, parmi {', '.join(all_genres)}: ")

user_genres = user_genres.split(',')
user_genres = ' '.join([genre.strip() for genre in user_genres])

print(f"Genres sélectionnés par l'utilisateur: {user_genres}")


Genres sélectionnés par l'utilisateur: Comedy


In [22]:
user_tfidf = tfidf.transform([user_genres])

similarity = cosine_similarity(user_tfidf, tfidf_matrix)

similarity_scores = similarity.flatten()

df_movies['similarity_score'] = similarity_scores

movies_sorted = df_movies.sort_values(by='similarity_score', ascending=False)

print(movies_sorted[['title', 'genres', 'similarity_score']].head(10))

                                      title                     genres  \
0                          Toy Story (1995)    genre_3 genre_4 genre_5   
1104                       Firestorm (1998)   genre_1 genre_2 genre_16   
1128               Chungking Express (1994)  genre_8 genre_13 genre_14   
1127  Heidi Fleiss: Hollywood Madam (1995)                     genre_7   
1126                Truman Show, The (1998)                    genre_8   
1125        Old Man and the Sea, The (1958)            genre_2 genre_8   
1124                  Innocents, The (1961)                   genre_16   
1123             Farewell to Arms, A (1932)          genre_14 genre_17   
1122      Last Time I Saw Paris, The (1954)                    genre_8   
1121         They Made Me a Criminal (1939)            genre_6 genre_8   

      similarity_score  
0                  0.0  
1104               0.0  
1128               0.0  
1127               0.0  
1126               0.0  
1125               0.0  
1124      

### Partie 4

In [25]:

user_id = 1

all_movie_ids = df_movies['movie_id'].tolist()
rated_movies = df_ratings[df_ratings['user_id'] == user_id]['movie_id'].tolist()

unrated_movies = [movie_id for movie_id in all_movie_ids if movie_id not in rated_movies]

predictions = [algo.predict(user_id, movie_id) for movie_id in unrated_movies]

predictions.sort(key=lambda x: x.est, reverse=True)

top_10_collab = predictions[:10]
top_10_movie_ids = [pred.iid for pred in top_10_collab]

print(f"Films recommandés par filtrage collaboratif pour l'utilisateur {user_id} : {top_10_movie_ids}")


Films recommandés par filtrage collaboratif pour l'utilisateur 1 : [1122, 1189, 1201, 1293, 1398, 1429, 1467, 1491, 1536, 1599]


In [26]:

user_genres = input(f"Veuillez entrer vos genres préférés, séparés par des virgules, parmi {', '.join(all_genres)}: ")
user_genres = user_genres.split(',')
user_genres = ' '.join([genre.strip() for genre in user_genres])

user_tfidf = tfidf.transform([user_genres])


filtered_movies = df_movies[df_movies['movie_id'].isin(top_10_movie_ids)]
similarity = cosine_similarity(user_tfidf, tfidf.transform(filtered_movies['genres']))

filtered_movies['similarity_score'] = similarity.flatten()

filtered_movies = filtered_movies.sort_values(by='similarity_score', ascending=False)

top_5_hybrid = filtered_movies[['title', 'genres', 'similarity_score']].head(5)

print("Top 5 des films recommandés via système hybride :")
print(top_5_hybrid)


Top 5 des films recommandés via système hybride :
                                           title  \
1121              They Made Me a Criminal (1939)   
1188                          Prefontaine (1997)   
1200  Marlene Dietrich: Shadow and Light (1996)    
1292                             Star Kid (1997)   
1397                                 Anna (1996)   

                                genres  similarity_score  
1121                   genre_6 genre_8               0.0  
1188                           genre_8               0.0  
1200                           genre_7               0.0  
1292  genre_2 genre_4 genre_9 genre_15               0.0  
1397                           genre_8               0.0  


C:\Users\leo-a\AppData\Local\Temp\ipykernel_79860\2779325029.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_movies['similarity_score'] = similarity.flatten()


In [27]:

all_genres = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 
              'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

user_genres = input(f"Veuillez entrer vos genres préférés, séparés par des virgules, parmi {', '.join(all_genres)}: ")

user_genres = [genre.strip() for genre in user_genres.split(',')]
user_genres = ' '.join(user_genres) 
print(f"Genres préférés de l'utilisateur : {user_genres}")


Genres préférés de l'utilisateur : Crime


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer()

tfidf_matrix = tfidf.fit_transform(df_movies['genres'])

user_tfidf = tfidf.transform([user_genres])

similarity = cosine_similarity(user_tfidf, tfidf_matrix)

df_movies['similarity_score'] = similarity.flatten()

movies_sorted = df_movies.sort_values(by='similarity_score', ascending=False)

top_5_genres_recommendations = movies_sorted[['title', 'genres', 'similarity_score']].head(5)
print("Top 5 films correspondant aux genres préférés de l'utilisateur :")
print(top_5_genres_recommendations)


Top 5 films correspondant aux genres préférés de l'utilisateur :
                                      title                     genres  \
0                          Toy Story (1995)    genre_3 genre_4 genre_5   
1104                       Firestorm (1998)   genre_1 genre_2 genre_16   
1128               Chungking Express (1994)  genre_8 genre_13 genre_14   
1127  Heidi Fleiss: Hollywood Madam (1995)                     genre_7   
1126                Truman Show, The (1998)                    genre_8   

      similarity_score  
0                  0.0  
1104               0.0  
1128               0.0  
1127               0.0  
1126               0.0  
